# TDSM v2 — Colab(T4)（UNSWテストしか無いときの自動分割版）
- Drive 内に **`UNSW_NB15_testing-set.csv` しか無い**場合でも、これを**層化して train/val を自動生成**します（既存の test はそのまま使用）。
- 以後は標準化→PCAベースライン→VIB(8–16D)→早期退出→AutoAttack→可視化→保存までワンパスです。

In [1]:
#@title GPU / ランタイム確認
!nvidia-smi || true
import torch, platform
print("PyTorch:", torch.__version__, "| CUDA available:", torch.cuda.is_available())
print("Python:", platform.python_version())

Sat Aug 30 17:36:24 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   63C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
#@title 依存導入（PyTorchはColab既定）
!pip -q install scikit-learn pandas pyarrow umap-learn shap tqdm matplotlib seaborn
!pip -q install "git+https://github.com/fra31/auto-attack.git"
print("Installed.")

  Preparing metadata (setup.py) ... done
Installed.


In [3]:
#@title 設定（乱数・ディレクトリ）
import os, random, numpy as np, torch
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DTYPE = torch.float16  # T4はFP16推奨
DRIVE_MOUNT = "/content/drive"
DRIVE_DIR = f"{DRIVE_MOUNT}/MyDrive/tdsm_v2"
print({"device": DEVICE, "drive_dir": DRIVE_DIR})

{'device': 'cuda', 'drive_dir': '/content/drive/MyDrive/tdsm_v2'}


In [4]:
#@title Driveマウント（安全な再マウント）
from google.colab import drive
import os, shutil

MNT = "/content/drive"
try:
    drive.flush_and_unmount()
except Exception:
    pass
if os.path.isdir(MNT) and os.listdir(MNT):
    shutil.rmtree(MNT)
os.makedirs(MNT, exist_ok=True)

drive.mount(MNT)
print("Mounted at:", MNT)

for d in ["data", "artifacts", "logs", "configs"]:
    os.makedirs(os.path.join(DRIVE_DIR, d), exist_ok=True)
print("Ready:", os.listdir(DRIVE_DIR))

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive
Mounted at: /content/drive
Ready: ['data', 'artifacts', 'logs', 'configs']


In [5]:
#@title Drive内を探索して train/val/test を検出（.parquet/.csv）
import os, json, pandas as pd
from collections import defaultdict

SEARCH_ROOTS = [
    os.path.join(DRIVE_DIR, "data"),
    DRIVE_DIR,
    "/content/drive/MyDrive",
]
ALLOWED_EXTS = (".parquet", ".csv")
ROLE_KEYS = {
    "train": ["train", "training", "trn"],
    "val":   ["val", "valid", "validation", "dev"],
    "test":  ["test", "tst", "eval"],
}

def _depth(p, root): return os.path.relpath(p, root).count(os.sep)
def _role_from_name(name):
    n = name.lower()
    for role, keys in ROLE_KEYS.items():
        if any(k in n for k in keys):
            return role
    return None

def scan_candidates(root, max_depth=5, max_files=5000):
    found = []
    seen = 0
    for dirpath, dirnames, filenames in os.walk(root):
        if _depth(dirpath, root) > max_depth:
            continue
        for fn in filenames:
            seen += 1
            if seen > max_files: break
            if fn.lower().endswith(ALLOWED_EXTS):
                role = _role_from_name(fn)
                found.append((role, os.path.join(dirpath, fn)))
        if seen > max_files: break
    return found

buckets = defaultdict(lambda: defaultdict(list))
for root in SEARCH_ROOTS:
    if not os.path.isdir(root): continue
    for role, path in scan_candidates(root):
        parent = os.path.dirname(path)
        buckets[parent][role].append(path)

def score_dir(d):
    roles = buckets[d]; have = set(k for k,v in roles.items() if len(v)>0)
    base = 0
    if "train" in have: base += 2
    if "test"  in have: base += 2
    if "val"   in have: base += 1
    name = d.lower()
    if any(k in name for k in ["unsw","cic","ids","rebuilt","quic","visquic","cesnet"]):
        base += 1
    return base

candidates = sorted(buckets.keys(), key=score_dir, reverse=True)
assert len(candidates) > 0, "Drive内に .parquet/.csv が見つかりませんでした。"
chosen = candidates[0]; roles = buckets[chosen]
def pick_first(role): return sorted(roles.get(role, []))[0] if roles.get(role) else None

train_path = pick_first("train")
val_path   = pick_first("val")
test_path  = pick_first("test")

print("候補ディレクトリ:", chosen)
print("train:", train_path)
print("val:  ", val_path)
print("test: ", test_path)

DATA_PATHS = {"train": train_path, "val": val_path, "test": test_path}
USE_SYNTHETIC = False

# 軽く保存（ログ用）
json.dump(DATA_PATHS, open(os.path.join(DRIVE_DIR,"configs","DATA_PATHS_detected.json"),"w"), indent=2, ensure_ascii=False)

候補ディレクトリ: /content/drive/MyDrive
train: None
val:   None
test:  /content/drive/MyDrive/UNSW_NB15_testing-set.csv


In [6]:
#@title （UNSW専用）testしか無い場合：UNSW_NB15_testing-set.csv から train/val を自動生成
import os, pandas as pd
from sklearn.model_selection import train_test_split

VAL_FRAC = 0.20  # testから val を切り出す割合（train:val=80:20）

def _load_df(path):
    return pd.read_parquet(path) if path.lower().endswith(".parquet") else pd.read_csv(path)

def _detect_or_make_label(df):
    # 優先順位: label/Label -> class/Class -> attack_cat (Normal=0, else 1)
    for cand in ["label","Label","class","Class","target","Target","y"]:
        if cand in df.columns:
            return cand, df[cand]
    if "attack_cat" in df.columns:
        return "label", (df["attack_cat"].astype(str).str.lower()!="normal").astype("int64")
    raise ValueError("ラベル列が見つかりません（label/Label/class/attack_catなど）。")

if DATA_PATHS.get("train") is None and DATA_PATHS.get("val") is None and DATA_PATHS.get("test") is not None:
    src = DATA_PATHS["test"]
    base = os.path.basename(src).lower()
    if "unsw" in base and "testing" in base:
        print("UNSWテストのみ検出：", src)
        df = _load_df(src)
        # ラベル確認（無ければ attack_cat から作る）
        label_col, series = _detect_or_make_label(df)
        if label_col != "label":
            df["label"] = series.values  # 統一
            label_col = "label"
        # stratified split: train/val（testは元ファイルをそのまま使用）
        tr, va = train_test_split(df, test_size=VAL_FRAC, random_state=42, stratify=df[label_col])
        parent = os.path.dirname(src)
        train_out = os.path.join(parent, f"auto_train_from_UNSWtest.csv")
        val_out   = os.path.join(parent, f"auto_val_from_UNSWtest.csv")
        tr.to_csv(train_out, index=False); va.to_csv(val_out, index=False)
        DATA_PATHS["train"] = train_out
        DATA_PATHS["val"]   = val_out
        print("生成しました:", train_out, val_out)
    else:
        print("testのみですがUNSWテスト名ではないため自動分割はスキップしました。")

print("最終 DATA_PATHS:", DATA_PATHS)

UNSWテストのみ検出： /content/drive/MyDrive/UNSW_NB15_testing-set.csv
生成しました: /content/drive/MyDrive/auto_train_from_UNSWtest.csv /content/drive/MyDrive/auto_val_from_UNSWtest.csv
最終 DATA_PATHS: {'train': '/content/drive/MyDrive/auto_train_from_UNSWtest.csv', 'val': '/content/drive/MyDrive/auto_val_from_UNSWtest.csv', 'test': '/content/drive/MyDrive/UNSW_NB15_testing-set.csv'}


In [7]:
#@title データローダ（数値特徴だけを自動選択）
import pandas as pd, numpy as np, os

def load_df(path):
    return pd.read_parquet(path) if path.lower().endswith(".parquet") else pd.read_csv(path)

assert DATA_PATHS["train"] is not None and DATA_PATHS["val"] is not None and DATA_PATHS["test"] is not None,     "DATA_PATHS が揃っていません（自動分割セルのログを確認してください）。"

df_train, df_val, df_test = (load_df(DATA_PATHS["train"]), load_df(DATA_PATHS["val"]), load_df(DATA_PATHS["test"]))

# ラベル列の特定（なければ 'label' を作ることも考慮）
def pick_label(df):
    for cand in ["label","Label","class","Class","target","Target","y"]:
        if cand in df.columns: return cand
    if "attack_cat" in df.columns: return "attack_cat"
    raise ValueError("ラベル列が見つかりません。")

label_col = pick_label(df_train)
if label_col != "label":
    if label_col == "attack_cat":
        for d in (df_train, df_val, df_test):
            d["label"] = (d["attack_cat"].astype(str).str.lower()!="normal").astype("int64")
    else:
        for d in (df_train, df_val, df_test):
            d["label"] = d[label_col].astype("int64")
    label_col = "label"

# 数値列のみを特徴に採用（ポート/フラグ/統計など）
num_cols = [c for c in df_train.select_dtypes(include=[np.number]).columns if c != "label"]
feature_cols = num_cols
print("features:", len(feature_cols), "| sample:", feature_cols[:8], "...")
print("Shapes:", df_train.shape, df_val.shape, df_test.shape, "| label:", label_col)
df_train.head(3)

features: 40 | sample: ['id', 'dur', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'sttl'] ...
Shapes: (65865, 45) (16467, 45) (82332, 45) | label: label


,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,46264,1.457374,tcp,http,FIN,10,16,814,9878,17.154142,...,1,1,0,0,1,2,1,0,Exploits,1
1,3575,0.654355,tcp,-,FIN,10,8,564,354,25.979782,...,1,1,0,0,0,1,1,0,Reconnaissance,1
2,43204,0.000002,udp,-,INT,2,0,1064,0,500000.001300,...,1,34,0,0,0,10,33,0,Normal,0


In [8]:
#@title 標準化→Tensor化
from sklearn.preprocessing import StandardScaler
import torch

scaler = StandardScaler().fit(df_train[feature_cols].values)

def to_tensors(df):
    X = scaler.transform(df[feature_cols].values).astype("float32")
    y = df["label"].astype("int64").values
    return torch.tensor(X), torch.tensor(y)

Xtr, ytr = to_tensors(df_train)
Xva, yva = to_tensors(df_val)
Xte, yte = to_tensors(df_test)

print(Xtr.shape, ytr.shape, Xva.shape, yva.shape)

torch.Size([65865, 40]) torch.Size([65865]) torch.Size([16467, 40]) torch.Size([16467])


In [9]:
#@title ベースライン（PCA→LogReg）
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score

pca = PCA(n_components=min(32, Xtr.shape[1])).fit(Xtr.numpy())
Ztr, Zva, Zte = pca.transform(Xtr.numpy()), pca.transform(Xva.numpy()), pca.transform(Xte.numpy())

clf = LogisticRegression(max_iter=1000, n_jobs=1).fit(Ztr, ytr.numpy())
proba_va = clf.predict_proba(Zva)[:,1]; proba_te = clf.predict_proba(Zte)[:,1]
print("Val AUROC/AUPRC:",
      round(roc_auc_score(yva.numpy(), proba_va),4),
      round(average_precision_score(yva.numpy(), proba_va),4))
print("Test AUROC/AUPRC:",
      round(roc_auc_score(yte.numpy(), proba_te),4),
      round(average_precision_score(yte.numpy(), proba_te),4))

Val AUROC/AUPRC: 0.9846 0.9868
Test AUROC/AUPRC: 0.9844 0.9867


In [10]:
#@title TDSM-MVP（VIB 8–16D＋早期退出）学習
import torch, torch.nn as nn, torch.nn.functional as F, time
from torch.utils.data import TensorDataset, DataLoader

LATENT_D = 16
BATCH = 512
EPOCHS = 5
BETA = 1e-2
DELTA = 0.90

train_loader = DataLoader(TensorDataset(Xtr, ytr), batch_size=BATCH, shuffle=True, num_workers=2, pin_memory=True, persistent_workers=True)
val_loader   = DataLoader(TensorDataset(Xva, yva), batch_size=BATCH, shuffle=False, num_workers=2, pin_memory=True, persistent_workers=True)
test_loader  = DataLoader(TensorDataset(Xte, yte), batch_size=BATCH, shuffle=False, num_workers=2, pin_memory=True, persistent_workers=True)

class VIBNet(nn.Module):
    def __init__(self, in_dim, z_dim, n_classes=2):
        super().__init__()
        self.enc = nn.Sequential(nn.Linear(in_dim, 256), nn.ReLU(), nn.Linear(256, 128), nn.ReLU())
        self.mu = nn.Linear(128, z_dim); self.logvar = nn.Linear(128, z_dim)
        self.head_early = nn.Linear(z_dim, n_classes)
        self.head_final = nn.Linear(z_dim, n_classes)
    def reparam(self, mu, logvar):
        std = torch.exp(0.5*logvar); eps = torch.randn_like(std)
        return mu + eps * std
    def forward(self, x, early_threshold=None):
        h = self.enc(x)
        mu, logvar = self.mu(h), self.logvar(h)
        logits_early = self.head_early(mu)
        if early_threshold is not None:
            conf = F.softmax(logits_early, dim=1).max(dim=1).values
            use_early = conf >= early_threshold
        else:
            use_early = torch.zeros(x.size(0), dtype=torch.bool, device=x.device)
        z = self.reparam(mu, logvar); logits_final = self.head_final(z)
        return logits_early, logits_final, mu, logvar, use_early

def vib_loss(logits, y, mu, logvar, beta=BETA):
    ce = F.cross_entropy(logits, y)
    kl = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp(), dim=1).mean()
    return ce + beta * kl, ce, kl

model = VIBNet(in_dim=Xtr.shape[1], z_dim=LATENT_D).to(DEVICE)
opt = torch.optim.AdamW(model.parameters(), lr=1e-3)
scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE=="cuda"))

def evaluate(loader, use_early=True):
    model.eval(); all_p, all_y = [], []
    with torch.no_grad(), torch.cuda.amp.autocast(enabled=(DEVICE=="cuda"), dtype=torch.float16):
        for xb, yb in loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            le, lf, mu, logvar, use_e = model(xb, early_threshold=DELTA if use_early else None)
            logits = torch.where(use_e.unsqueeze(1), le, lf)
            prob1 = F.softmax(logits, dim=1)[:,1]
            all_p.append(prob1.detach().cpu()); all_y.append(yb.cpu())
    import numpy as np
    p = torch.cat(all_p).numpy(); y = torch.cat(all_y).numpy()
    from sklearn.metrics import roc_auc_score, average_precision_score
    return float(roc_auc_score(y, p)), float(average_precision_score(y, p))

best = -1.0
for ep in range(1, EPOCHS+1):
    model.train(); import time; t0=time.time()
    for xb, yb in train_loader:
        xb, yb = xb.to(DEVICE), yb.to(DEVICE)
        opt.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(enabled=(DEVICE=="cuda"), dtype=torch.float16):
            le, lf, mu, logvar, use_e = model(xb, early_threshold=DELTA)
            loss, ce, kl = vib_loss(lf, yb, mu, logvar, beta=BETA)
        scaler.scale(loss).backward(); scaler.step(opt); scaler.update()
    val_roc, val_ap = evaluate(val_loader, use_early=True)
    print(f"[EP{ep}] val AUROC={val_roc:.4f} AP={val_ap:.4f} | time={time.time()-t0:.1f}s")
    if val_roc > best:
        best = val_roc
        torch.save(model.state_dict(), os.path.join(DRIVE_DIR,"artifacts","vib_best.pt"))

/tmp/ipython-input-3335009135.py:44: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE=="cuda"))
/tmp/ipython-input-3335009135.py:66: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(DEVICE=="cuda"), dtype=torch.float16):
/tmp/ipython-input-3335009135.py:48: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast(enabled=(DEVICE=="cuda"), dtype=torch.float16):


[EP1] val AUROC=0.9872 AP=0.9881 | time=2.5s


/tmp/ipython-input-3335009135.py:66: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(DEVICE=="cuda"), dtype=torch.float16):
/tmp/ipython-input-3335009135.py:48: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast(enabled=(DEVICE=="cuda"), dtype=torch.float16):
/tmp/ipython-input-3335009135.py:66: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(DEVICE=="cuda"), dtype=torch.float16):


[EP2] val AUROC=0.9925 AP=0.9929 | time=1.0s


/tmp/ipython-input-3335009135.py:48: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast(enabled=(DEVICE=="cuda"), dtype=torch.float16):
/tmp/ipython-input-3335009135.py:66: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(DEVICE=="cuda"), dtype=torch.float16):


[EP3] val AUROC=0.9961 AP=0.9964 | time=1.0s


/tmp/ipython-input-3335009135.py:48: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast(enabled=(DEVICE=="cuda"), dtype=torch.float16):
/tmp/ipython-input-3335009135.py:66: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(DEVICE=="cuda"), dtype=torch.float16):


[EP4] val AUROC=0.9974 AP=0.9974 | time=1.0s
[EP5] val AUROC=0.9979 AP=0.9979 | time=1.2s


/tmp/ipython-input-3335009135.py:48: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast(enabled=(DEVICE=="cuda"), dtype=torch.float16):


In [11]:
#@title 推論レイテンシ & 早期退出
import time, numpy as np, torch.nn.functional as F, os, torch
model.load_state_dict(torch.load(os.path.join(DRIVE_DIR,"artifacts","vib_best.pt"), map_location=DEVICE))
model.eval()

def latency_test(loader, use_early=True, iters=50):
    for xb, yb in loader:
        xb = xb.to(DEVICE)
        with torch.no_grad(), torch.cuda.amp.autocast(enabled=(DEVICE=='cuda'), dtype=torch.float16):
            _ = model(xb, early_threshold=DELTA if use_early else None)
        break
    if DEVICE=='cuda': torch.cuda.synchronize()
    t=[]; exit_ratio=[]
    with torch.no_grad():
        for i,(xb,yb) in enumerate(loader):
            xb=xb.to(DEVICE)
            if DEVICE=='cuda': torch.cuda.synchronize()
            t0 = time.time()
            with torch.cuda.amp.autocast(enabled=(DEVICE=='cuda'), dtype=torch.float16):
                le, lf, mu, logvar, use_e = model(xb, early_threshold=DELTA if use_early else None)
            if DEVICE=='cuda': torch.cuda.synchronize()
            t.append(time.time()-t0)
            if use_early: exit_ratio.append(use_e.float().mean().item())
            if i>=iters: break
    mean_ms = float(np.mean(t)*1000); p95_ms = float(np.percentile(t,95)*1000)
    ex = float(np.mean(exit_ratio) if use_early and exit_ratio else 0.0)
    return mean_ms, p95_ms, ex

from torch.utils.data import TensorDataset, DataLoader
test_loader  = DataLoader(TensorDataset(Xte, yte), batch_size=512, shuffle=False, num_workers=2, pin_memory=True, persistent_workers=True)

ms_mean, ms_p95, exit_r = latency_test(test_loader, use_early=True)
print(f"Early-Exit: mean {ms_mean:.2f} ms | p95 {ms_p95:.2f} ms | exit_ratio {exit_r:.2%}")
ms_mean2, ms_p952, _ = latency_test(test_loader, use_early=False)
print(f"No-Exit:   mean {ms_mean2:.2f} ms | p95 {ms_p952:.2f} ms")

/tmp/ipython-input-1698503753.py:9: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast(enabled=(DEVICE=='cuda'), dtype=torch.float16):
/tmp/ipython-input-1698503753.py:19: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(DEVICE=='cuda'), dtype=torch.float16):


Early-Exit: mean 1.15 ms | p95 1.88 ms | exit_ratio 0.00%
No-Exit:   mean 1.16 ms | p95 2.18 ms


In [14]:
#@title AutoAttack（tabular/2値向け：APGD-CEのみ）
import torch, torch.nn as nn, torch.nn.functional as F
from autoattack import AutoAttack

model.eval()

# 決定論ラッパー：サンプリングなし（μ→head_early）
class DetWrapper(nn.Module):
    def __init__(self, mdl):
        super().__init__(); self.mdl = mdl
    def forward(self, x):
        h  = self.mdl.enc(x)
        mu = self.mdl.mu(h)
        return self.mdl.head_early(mu)

wrp = DetWrapper(model).to(DEVICE).eval()

Xaa, yaa = Xte.to(DEVICE), yte.to(DEVICE)

# APGD-CE のみ（tabular/2値向け）
adversary = AutoAttack(wrp, norm='L2', eps=0.25, version='custom')
adversary.attacks_to_run = ['apgd-ce']
adversary.seed = 0  # 再現性
# 速度/厳しさの好みに応じて
adversary.apgd.n_iter = 100
adversary.apgd.n_restarts = 1

# クリーン精度
with torch.no_grad():
    clean_pred = torch.softmax(wrp(Xaa),1).argmax(1)
clean_acc = (clean_pred==yaa).float().mean().item()

# 逆例生成（ここはno_gradにしない）
x_adv = adversary.run_standard_evaluation(Xaa, yaa, bs=256)

# ロバスト精度
with torch.no_grad():
    adv_pred = torch.softmax(wrp(x_adv),1).argmax(1)
robust_acc = (adv_pred==yaa).float().mean().item()

print({"clean_acc": round(clean_acc,4), "robust_acc": round(robust_acc,4)})


using custom version including apgd-ce.
initial accuracy: 98.68%
apgd-ce - 1/318 - 121 out of 256 successfully perturbed
apgd-ce - 2/318 - 175 out of 256 successfully perturbed
apgd-ce - 3/318 - 212 out of 256 successfully perturbed
apgd-ce - 4/318 - 199 out of 256 successfully perturbed
apgd-ce - 5/318 - 195 out of 256 successfully perturbed
apgd-ce - 6/318 - 198 out of 256 successfully perturbed
apgd-ce - 7/318 - 191 out of 256 successfully perturbed
apgd-ce - 8/318 - 197 out of 256 successfully perturbed
apgd-ce - 9/318 - 211 out of 256 successfully perturbed
apgd-ce - 10/318 - 213 out of 256 successfully perturbed
apgd-ce - 11/318 - 190 out of 256 successfully perturbed
apgd-ce - 12/318 - 214 out of 256 successfully perturbed
apgd-ce - 13/318 - 221 out of 256 successfully perturbed
apgd-ce - 14/318 - 220 out of 256 successfully perturbed
apgd-ce - 15/318 - 227 out of 256 successfully perturbed
apgd-ce - 16/318 - 223 out of 256 successfully perturbed
apgd-ce - 17/318 - 216 out of 25

In [17]:
#@title TDSM-MVP（VIB 8–16D＋早期退出）学習【修正版】
import torch, torch.nn as nn, torch.nn.functional as F, time
from torch.utils.data import TensorDataset, DataLoader

LATENT_D = 16
BATCH = 512
EPOCHS = 5
BETA = 1e-2
DELTA = 0.90

train_loader = DataLoader(TensorDataset(Xtr, ytr), batch_size=BATCH, shuffle=True, num_workers=2, pin_memory=True, persistent_workers=True)
val_loader   = DataLoader(TensorDataset(Xva, yva), batch_size=BATCH, shuffle=False, num_workers=2, pin_memory=True, persistent_workers=True)
test_loader  = DataLoader(TensorDataset(Xte, yte), batch_size=BATCH, shuffle=False, num_workers=2, pin_memory=True, persistent_workers=True)

class VIBNet(nn.Module):
    def __init__(self, in_dim, z_dim, n_classes=2):
        super().__init__()
        self.enc = nn.Sequential(nn.Linear(in_dim, 256), nn.ReLU(), nn.Linear(256, 128), nn.ReLU())
        self.mu = nn.Linear(128, z_dim); self.logvar = nn.Linear(128, z_dim)
        self.head_early = nn.Linear(z_dim, n_classes)
        self.head_final = nn.Linear(z_dim, n_classes)
    def reparam(self, mu, logvar):
        std = torch.exp(0.5*logvar); eps = torch.randn_like(std)
        return mu + eps * std
    def forward(self, x, early_threshold=None):
        h = self.enc(x)
        mu, logvar = self.mu(h), self.logvar(h)
        logits_early = self.head_early(mu)
        if early_threshold is not None:
            conf = F.softmax(logits_early, dim=1).max(dim=1).values
            use_early = conf >= early_threshold
        else:
            use_early = torch.zeros(x.size(0), dtype=torch.bool, device=x.device)
        z = self.reparam(mu, logvar); logits_final = self.head_final(z)
        return logits_early, logits_final, mu, logvar, use_early

def vib_loss(logits, y, mu, logvar, beta=BETA):
    ce = F.cross_entropy(logits, y)
    kl = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp(), dim=1).mean()
    return ce + beta * kl, ce, kl

model = VIBNet(in_dim=Xtr.shape[1], z_dim=LATENT_D).to(DEVICE)

opt = torch.optim.AdamW(model.parameters(), lr=1e-3)
# ★ GradScaler は amp_scaler という別名にする（前処理の StandardScaler と衝突させない）
amp_scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE=="cuda"))

def evaluate(loader, use_early=True):
    model.eval(); all_p, all_y = [], []
    with torch.no_grad(), torch.amp.autocast(device_type="cuda", enabled=(DEVICE=="cuda"), dtype=torch.float16):
        for xb, yb in loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            le, lf, mu, logvar, use_e = model(xb, early_threshold=DELTA if use_early else None)
            logits = torch.where(use_e.unsqueeze(1), le, lf)
            prob1 = F.softmax(logits, dim=1)[:,1]
            all_p.append(prob1.detach().cpu()); all_y.append(yb.cpu())
    p = torch.cat(all_p).numpy(); y = torch.cat(all_y).numpy()
    from sklearn.metrics import roc_auc_score, average_precision_score
    return float(roc_auc_score(y, p)), float(average_precision_score(y, p))

best = -1.0
for ep in range(1, EPOCHS+1):
    model.train(); t0=time.time()
    for xb, yb in train_loader:
        xb, yb = xb.to(DEVICE), yb.to(DEVICE)
        opt.zero_grad(set_to_none=True)
        with torch.amp.autocast(device_type="cuda", enabled=(DEVICE=="cuda"), dtype=torch.float16):
            le, lf, mu, logvar, use_e = model(xb, early_threshold=DELTA)
            loss, ce, kl = vib_loss(lf, yb, mu, logvar, beta=BETA)
        amp_scaler.scale(loss).backward()
        amp_scaler.step(opt); amp_scaler.update()
    val_roc, val_ap = evaluate(val_loader, use_early=True)
    print(f"[EP{ep}] val AUROC={val_roc:.4f} AP={val_ap:.4f} | time={time.time()-t0:.1f}s")
    if val_roc > best:
        best = val_roc
        import os
        torch.save(model.state_dict(), os.path.join(DRIVE_DIR, "artifacts", "vib_best.pt"))


/tmp/ipython-input-1954849698.py:46: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  amp_scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE=="cuda"))


[EP1] val AUROC=0.9868 AP=0.9874 | time=2.3s
[EP2] val AUROC=0.9935 AP=0.9944 | time=1.1s
[EP3] val AUROC=0.9969 AP=0.9971 | time=1.1s
[EP4] val AUROC=0.9976 AP=0.9977 | time=1.1s
[EP5] val AUROC=0.9973 AP=0.9971 | time=1.2s


In [18]:
#@title 成果物の保存
import joblib, json, os, torch
joblib.dump(scaler, os.path.join(DRIVE_DIR, "artifacts", "scaler.pkl"))
torch.save(model.state_dict(), os.path.join(DRIVE_DIR, "artifacts", "vib_best.pt"))
conf = dict(seed=int(SEED), latent=int(LATENT_D), beta=float(BETA), delta=float(DELTA), features=len(feature_cols),
            data_paths=DATA_PATHS)
import json
json.dump(conf, open(os.path.join(DRIVE_DIR, "configs", "run.json"), "w"), indent=2, ensure_ascii=False)
print("Saved:", os.listdir(os.path.join(DRIVE_DIR,"artifacts")))

Saved: ['vib_best.pt', 'scaler.pkl']


In [20]:
#@title 🔁 フル自己完結：自動レポート生成セル（セットアップ込み）
import os, json, time, datetime, numpy as np, pandas as pd, matplotlib.pyplot as plt
import torch, torch.nn as nn, torch.nn.functional as F

# ================= ユーザ設定（必要なら変更） =================
PROJ_DIR_DEFAULT = "/content/drive/MyDrive/tdsm_v2"
N_BINS = 15
THRESH_GRID = np.linspace(0.01, 0.99, 99)
EPS_LIST = [0.05, 0.10, 0.15, 0.20, 0.25]  # AutoAttack L2
AA_ITERS = 100
AA_RESTARTS = 1
RUN_EPS_SWEEP = True
# =========================================================

# ---- Colab Drive を未マウントならマウント ----
try:
    from google.colab import drive  # noqa
    need_mount = not os.path.isdir("/content/drive/MyDrive")
    if need_mount:
        drive.mount("/content/drive")
except Exception:
    pass  # Colab以外

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
PROJ_DIR = PROJ_DIR_DEFAULT
os.makedirs(PROJ_DIR, exist_ok=True)
CFG_DIR  = os.path.join(PROJ_DIR, "configs")
ART_DIR  = os.path.join(PROJ_DIR, "artifacts")
DATA_DIR = os.path.join(PROJ_DIR, "data")
os.makedirs(CFG_DIR, exist_ok=True); os.makedirs(ART_DIR, exist_ok=True); os.makedirs(DATA_DIR, exist_ok=True)

stamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
REPORT_DIR = os.path.join(PROJ_DIR, "reports", stamp)
os.makedirs(REPORT_DIR, exist_ok=True)
print("REPORT_DIR:", REPORT_DIR)

# ---- DATA_PATHS を読み込み or 推定 ----
def load_json_if(p):
    return json.load(open(p)) if os.path.isfile(p) else None

DATA_PATHS = load_json_if(os.path.join(CFG_DIR, "DATA_PATHS.json")) or \
             load_json_if(os.path.join(CFG_DIR, "DATA_PATHS_detected.json"))

if not DATA_PATHS:
    # よくある配置から推測（UNSWの自動分割名を優先）
    cand_test = [
        os.path.join(DATA_DIR, "UNSW_NB15_testing-set.csv"),
        "/content/drive/MyDrive/UNSW_NB15_testing-set.csv"
    ]
    test_p = next((p for p in cand_test if os.path.isfile(p)), None)
    DATA_PATHS = {
        "train": os.path.join(DATA_DIR, "auto_train_from_UNSWtest.csv") if os.path.isfile(os.path.join(DATA_DIR,"auto_train_from_UNSWtest.csv")) else None,
        "val":   os.path.join(DATA_DIR, "auto_val_from_UNSWtest.csv")   if os.path.isfile(os.path.join(DATA_DIR,"auto_val_from_UNSWtest.csv")) else None,
        "test":  test_p
    }
print("DATA_PATHS:", DATA_PATHS)

# ---- データ読み込み（数値列のみ）＆ label 整備 ----
def load_df(path):
    if path is None: return None
    return pd.read_parquet(path) if path.lower().endswith(".parquet") else pd.read_csv(path)

def ensure_label(df):
    if df is None: return None
    for cand in ["label","Label","class","Class","target","Target","y"]:
        if cand in df.columns:
            if cand != "label":
                df["label"] = df[cand].astype("int64")
            return df
    if "attack_cat" in df.columns:
        df["label"] = (df["attack_cat"].astype(str).str.lower()!="normal").astype("int64")
        return df
    raise ValueError("ラベル列が見つかりません（label/class/attack_cat等）")

df_train = ensure_label(load_df(DATA_PATHS.get("train")))
df_val   = ensure_label(load_df(DATA_PATHS.get("val")))
df_test  = ensure_label(load_df(DATA_PATHS.get("test")))
assert df_test is not None, "test データが見つかりません。"

def pick_numeric_cols(df):
    import numpy as np
    return [c for c in df.select_dtypes(include=[np.number]).columns if c!="label"]

feature_cols = pick_numeric_cols(df_train if df_train is not None else df_test)
print("Feature dim:", len(feature_cols))

# ---- スケーラ（artifacts/scaler.pkl を優先）----
import joblib
scaler_path = os.path.join(ART_DIR, "scaler.pkl")
if os.path.isfile(scaler_path):
    scaler = joblib.load(scaler_path)
else:
    from sklearn.preprocessing import StandardScaler
    fit_df = df_train if df_train is not None else df_test
    scaler = StandardScaler().fit(fit_df[feature_cols].values)
    joblib.dump(scaler, scaler_path)

def to_tensors(df):
    X = scaler.transform(df[feature_cols].values).astype("float32")
    y = df["label"].astype("int64").values
    return torch.tensor(X), torch.tensor(y)

Xtr, ytr = (to_tensors(df_train) if df_train is not None else (None, None))
Xva, yva = (to_tensors(df_val)   if df_val   is not None else (None, None))
Xte, yte = to_tensors(df_test)

# ---- モデル定義＆ロード（VIBNet）----
class VIBNet(nn.Module):
    def __init__(self, in_dim, z_dim=16, n_classes=2):
        super().__init__()
        self.enc = nn.Sequential(nn.Linear(in_dim, 256), nn.ReLU(), nn.Linear(256, 128), nn.ReLU())
        self.mu = nn.Linear(128, z_dim); self.logvar = nn.Linear(128, z_dim)
        self.head_early = nn.Linear(z_dim, n_classes)
        self.head_final = nn.Linear(z_dim, n_classes)
    def reparam(self, mu, logvar):
        std = torch.exp(0.5*logvar); eps = torch.randn_like(std)
        return mu + eps * std
    def logits_det(self, x):  # 決定論：μ→early head
        h  = self.enc(x); mu = self.mu(h); return self.head_early(mu)
    def forward(self, x, early_threshold=None):
        h = self.enc(x)
        mu, logvar = self.mu(h), self.logvar(h)
        logits_early = self.head_early(mu)
        use_early = torch.zeros(x.size(0), dtype=torch.bool, device=x.device) if early_threshold is None \
                    else (F.softmax(logits_early, dim=1).max(dim=1).values >= early_threshold)
        z = self.reparam(mu, logvar); logits_final = self.head_final(z)
        return logits_early, logits_final, mu, logvar, use_early

# latent の既定値（run.json があれば拾う）
LATENT_D = 16
cfg_path = os.path.join(CFG_DIR, "run.json")
if os.path.isfile(cfg_path):
    try:
        cfg = json.load(open(cfg_path))
        LATENT_D = int(cfg.get("latent", LATENT_D))
    except Exception:
        pass

model = VIBNet(in_dim=len(feature_cols), z_dim=LATENT_D).to(DEVICE)
ckpt = os.path.join(ART_DIR, "vib_best.pt")
assert os.path.isfile(ckpt), f"モデル重みが見つかりません: {ckpt}"
model.load_state_dict(torch.load(ckpt, map_stage=DEVICE) if hasattr(torch, "load") else torch.load(ckpt, map_location=DEVICE))
model.eval()

# ---- 基本メトリクス（metrics.json が無ければ test だけでも作る）----
from sklearn.metrics import (precision_recall_fscore_support, accuracy_score,
                             roc_auc_score, average_precision_score,
                             roc_curve, precision_recall_curve, confusion_matrix)

def eval_split_logits(X, y, name="split", thr=0.5):
    with torch.no_grad():
        logits = model.logits_det(X.to(DEVICE))
        proba1 = torch.softmax(logits,1)[:,1].cpu().numpy()
    yt = y.numpy()
    yhat = (proba1 >= thr).astype(int)
    pr, rc, f1, _ = precision_recall_fscore_support(yt, yhat, average="binary", zero_division=0)
    acc = accuracy_score(yt, yhat)
    roc = roc_auc_score(yt, proba1)
    ap  = average_precision_score(yt, proba1)
    cm  = confusion_matrix(yt, yhat)
    return {"name": name, "acc": float(acc), "precision": float(pr), "recall": float(rc),
            "f1": float(f1), "auroc": float(roc), "auprc": float(ap), "cm": cm.tolist(),
            "proba": proba1, "y_true": yt}

metrics = []
if Xtr is not None: metrics.append(eval_split_logits(Xtr, ytr, "train"))
if Xva is not None: metrics.append(eval_split_logits(Xva, yva, "val"))
metrics.append(eval_split_logits(Xte, yte, "test"))

# 保存（pred_*.csv & metrics.json）
for m in metrics:
    pd.DataFrame({"y_true": m["y_true"], "proba1": m["proba"]}).to_csv(os.path.join(REPORT_DIR, f"pred_{m['name']}.csv"), index=False)
json.dump([{k:v for k,v in m.items() if k not in ("proba","y_true")} for m in metrics],
          open(os.path.join(REPORT_DIR, "metrics.json"), "w"), indent=2)

# ---- ここから自動レポート生成（ROC/PR, しきい値, ECE, εスイープ）----
y_true = metrics[-1]["y_true"]; proba = metrics[-1]["proba"]

# ROC/PR
fpr, tpr, _ = roc_curve(y_true, proba)
prec, rec, _ = precision_recall_curve(y_true, proba)
plt.figure(figsize=(4.5,4)); plt.plot(fpr,tpr); plt.plot([0,1],[0,1],'--'); plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title("ROC (test)")
plt.tight_layout(); plt.savefig(os.path.join(REPORT_DIR,"roc_test.png"), dpi=150); plt.show()
plt.figure(figsize=(4.5,4)); plt.plot(rec,prec); plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title("PR (test)")
plt.tight_layout(); plt.savefig(os.path.join(REPORT_DIR,"pr_test.png"), dpi=150); plt.show()

# しきい値スイープ（F1最大）
def metrics_at_threshold(y, p, thr):
    yhat = (p >= thr).astype(int)
    pr, rc, f1, _ = precision_recall_fscore_support(y, yhat, average="binary", zero_division=0)
    acc = accuracy_score(y, yhat)
    roc = roc_auc_score(y, p)
    ap  = average_precision_score(y, p)
    cm  = confusion_matrix(y, yhat)
    return dict(threshold=float(thr), acc=float(acc), precision=float(pr), recall=float(rc),
                f1=float(f1), auroc=float(roc), auprc=float(ap), cm=cm.tolist())

curves = [metrics_at_threshold(y_true, proba, t) for t in THRESH_GRID]
best = max(curves, key=lambda d: d["f1"])
plt.figure(figsize=(5,3.5))
plt.plot([c["threshold"] for c in curves], [c["f1"] for c in curves], label="F1")
plt.plot([c["threshold"] for c in curves], [c["precision"] for c in curves], label="Precision", alpha=0.7)
plt.plot([c["threshold"] for c in curves], [c["recall"] for c in curves], label="Recall",   alpha=0.7)
plt.axvline(best["threshold"], ls="--", label=f"best={best['threshold']:.2f}")
plt.legend(); plt.xlabel("Threshold"); plt.ylabel("Score"); plt.title("Threshold sweep (test)")
plt.tight_layout(); plt.savefig(os.path.join(REPORT_DIR,"threshold_sweep.png"), dpi=150); plt.show()
json.dump({"best": best, "curve": curves}, open(os.path.join(REPORT_DIR,"thresholds.json"),"w"), indent=2)

# 校正（ECE; 分位ビニング）
def ece_quantile(y, p, n_bins=15):
    qs = np.unique(np.quantile(p, np.linspace(0,1,n_bins+1)))
    if len(qs) < 3: qs = np.linspace(0,1,n_bins+1)
    idx = np.digitize(p, qs[1:-1], right=True)
    ece=0.0; xs=[]; ys=[]; bins=[]
    for b in range(len(qs)-1):
        mask = (idx==b)
        if not np.any(mask):
            bins.append(dict(bin=b, size=0)); continue
        conf = p[mask].mean(); acc = (y[mask]==(p[mask]>=0.5)).mean(); w = mask.mean()
        ece += w*abs(acc-conf)
        xs.append(conf); ys.append(acc)
        bins.append(dict(bin=b, lower=float(qs[b]), upper=float(qs[b+1]), size=int(mask.sum()),
                         conf=float(conf), acc=float(acc), weight=float(w)))
    return float(ece), xs, ys, bins, qs.tolist()

ece, xs, ys, bins, qs = ece_quantile(y_true, proba, n_bins=N_BINS)
plt.figure(figsize=(4.5,4)); plt.plot([0,1],[0,1],'--', lw=1); plt.scatter(xs, ys, s=25)
plt.xlabel("Predicted probability"); plt.ylabel("Empirical positive rate")
plt.title(f"Reliability diagram (test) | ECE={ece:.3f}")
plt.tight_layout(); plt.savefig(os.path.join(REPORT_DIR,"calibration_test.png"), dpi=150); plt.show()
json.dump({"n_bins": N_BINS, "ece": ece, "quantiles": qs, "bins": bins},
          open(os.path.join(REPORT_DIR, "ece.json"), "w"), indent=2)

# 早期退出率（DELTAは run.json があれば使用）
DELTA = 0.90
if os.path.isfile(cfg_path):
    try:
        cfg = json.load(open(cfg_path)); DELTA = float(cfg.get("delta", DELTA))
    except Exception: pass
from torch.utils.data import TensorDataset, DataLoader
test_loader = DataLoader(TensorDataset(Xte, yte), batch_size=512, shuffle=False)
use_early_all = []
with torch.no_grad():
    for xb, yb in test_loader:
        xb = xb.to(DEVICE)
        le, lf, mu, logvar, use_e = model(xb, early_threshold=DELTA)
        use_early_all.append(use_e.cpu())
ee_ratio = float(torch.cat(use_early_all).float().mean().item())
json.dump({"delta": DELTA, "early_ratio_test": ee_ratio}, open(os.path.join(REPORT_DIR,"early_exit.json"),"w"), indent=2)

# εスイープ（APGD-CE; tabular/2値）
rob_curve = []
if RUN_EPS_SWEEP:
    try:
        from autoattack import AutoAttack
        class DetWrapper(nn.Module):
            def __init__(self, mdl): super().__init__(); self.mdl = mdl
            def forward(self, x): return self.mdl.logits_det(x)
        wrp = DetWrapper(model).to(DEVICE).eval()
        Xaa, yaa = Xte.to(DEVICE), yte.to(DEVICE)
        with torch.no_grad():
            clean_acc = (torch.softmax(wrp(Xaa),1).argmax(1)==yaa).float().mean().item()
        for eps in EPS_LIST:
            aa = AutoAttack(wrp, norm='L2', eps=eps, version='custom')
            aa.attacks_to_run = ['apgd-ce']
            aa.apgd.n_iter = AA_ITERS; aa.apgd.n_restarts = AA_RESTARTS
            x_adv = aa.run_standard_evaluation(Xaa, yaa, bs=256)
            with torch.no_grad():
                rob = (torch.softmax(wrp(x_adv),1).argmax(1)==yaa).float().mean().item()
            rob_curve.append({"eps_L2": float(eps), "robust_acc": float(rob)})
        json.dump({"clean_acc": float(clean_acc), "curve": rob_curve,
                   "iters": AA_ITERS, "restarts": AA_RESTARTS},
                  open(os.path.join(REPORT_DIR,"robust_curve.json"),"w"), indent=2)
        plt.figure(figsize=(4.5,4)); plt.plot([d["eps_L2"] for d in rob_curve],
                                              [d["robust_acc"] for d in rob_curve], marker="o")
        plt.xlabel("L2 epsilon"); plt.ylabel("Robust accuracy"); plt.title("AutoAttack APGD-CE (test)")
        plt.tight_layout(); plt.savefig(os.path.join(REPORT_DIR,"robust_curve.png"), dpi=150); plt.show()
    except Exception as e:
        print("AutoAttack（εスイープ）をスキップ:", e)

# Markdown レポート
def fmt(v): return f"{v:.4f}" if isinstance(v, float) else str(v)
metrics_list = json.load(open(os.path.join(REPORT_DIR, "metrics.json")))
lines = ["# TDSM v2 — AUTO REPORT", ""]
lines.append("## 1. Overall metrics")
for m in metrics_list:
    lines.append(f"### {m['name']}")
    lines.append(f"- Acc={fmt(m['acc'])} | Precision={fmt(m['precision'])} | Recall={fmt(m['recall'])} | F1={fmt(m['f1'])}")
    lines.append(f"- AUROC={fmt(m['auroc'])} | AUPRC={fmt(m['auprc'])}")
    lines.append(f"- Confusion={m['cm']}")
    lines.append("")
lines.append("## 2. Threshold optimization (test)")
lines.append(f"- Best threshold (max F1): **{best['threshold']:.2f}**")
lines.append(f"- At best thr: Acc={fmt(best['acc'])} | P={fmt(best['precision'])} | R={fmt(best['recall'])} | F1={fmt(best['f1'])}")
lines.append(f"- AUROC={fmt(best['auroc'])} | AUPRC={fmt(best['auprc'])}")
lines.append("- Figures: `threshold_sweep.png`, `roc_test.png`, `pr_test.png`")
lines.append("")
lines.append("## 3. Calibration")
lines.append(f"- ECE (quantile {N_BINS} bins): **{ece:.3f}**")
lines.append("- Figure: `calibration_test.png`")
lines.append("")
lines.append("## 4. Early-Exit")
lines.append(f"- delta={DELTA}, early_ratio_test={ee_ratio:.3f}")
lines.append("")
rc_path = os.path.join(REPORT_DIR, "robust_curve.json")
if os.path.isfile(rc_path):
    aa = json.load(open(rc_path))
    lines.append("## 5. Robustness (AutoAttack APGD-CE, L2)")
    lines.append(f"- clean_acc={aa['clean_acc']:.3f}")
    for d in aa["curve"]:
        lines.append(f"  - eps={d['eps_L2']:.2f} → robust_acc={d['robust_acc']:.3f}")
    lines.append("- Figure: `robust_curve.png`")
    lines.append("")

auto_md = os.path.join(REPORT_DIR, "REPORT_AUTO.md")
with open(auto_md, "w") as f:
    f.write("\n".join(lines))

print("\n✅ Auto report written to:", auto_md)
print("📁 Folder:", REPORT_DIR)


REPORT_DIR: /content/drive/MyDrive/tdsm_v2/reports/20250830_183656
DATA_PATHS: {'train': None, 'val': None, 'test': '/content/drive/MyDrive/UNSW_NB15_testing-set.csv'}
Feature dim: 40


AttributeError: 'GradScaler' object has no attribute 'transform'